# Ground Rydberg Dynamics

We will use `GenStateChain` to simulate some Ground-Rydberg Hamiltonian Dynamics. We will call state 0 the ground state and state 1 the excited state.

We import the module and set a chain of 2 sites.

In [1]:
import GenStateChain as GSC
import numpy as np
GSC.set_n([2,2])

Now, we need to create our driven Rydberg Hamiltonian:

$$ H = \frac{\Omega}{2}\sum_{i = 1}^3\sigma_i^x - \Delta \sum_{i=1}^3 n_i + \sum_{i < j} \frac{C_6}{r^6} n_i n_j $$

where $ n_i = |r_i\rangle\langle r_i | $ 

In [7]:
# Create the Rabi drive term and detuning term
omega = 2 * np.pi * 1
delta = 0
op = GSC.Operators.zero()

for i in range(2):
    op = op + omega / 2 * GSC.Operators.sigmax([i], [[0,1]]) - delta * GSC.Operators.proj(GSC.create_state_list([i],1)) # The delta multiplies a projector of state 1 on site i

# Lastly we add the interaction term. Here we use an operator from the library.
coords = [[0,0],[1,0]] # two sites, 1 unit apart
C6 = 1
states = 1 # The interacting state is state 1. 
op = op + GSC.Operators.PairInteraction(coords, C6, states)

In [8]:
op

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.         3.14159265 3.14159265 0.        ]
 [3.14159265 0.         0.         3.14159265]
 [3.14159265 0.         0.         3.14159265]
 [0.         3.14159265 3.14159265 1.        ]]